# **Trabajo unidad 4: Clasificación de gestos de manos**
# INFO257 Inteligencia Artificial

**Integrantes**: *Patricio Canales*, *Eleazar Vásquez*

# **Descarga del dataset**

In [ ]:
!gdown "https://drive.google.com/u/0/uc?export=download&confirm=trOT&id=1m9fKMYpUX24sB9PijXq2g54EBxe2W-eO"

Downloading...
From: https://drive.google.com/u/0/uc?export=download&confirm=trOT&id=1m9fKMYpUX24sB9PijXq2g54EBxe2W-eO
To: /content/gestos.zip
148MB [00:01, 85.4MB/s]


In [ ]:
!unzip -q gestos.zip

In [ ]:
import torch
from torchvision.datasets import ImageFolder
from torchvision import transforms 
from torch.utils.data import DataLoader
import numpy as np
from torchvision import models

In [ ]:
if torch.cuda.is_available():
  display(torch.cuda.get_device_name(0))

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

'Tesla T4'

cuda


# **Dataloaders**

In [ ]:
train_transforms = transforms.Compose([transforms.Resize(255),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

valid_transforms = transforms.Compose([transforms.Resize(255),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])


test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])


train_dataset = ImageFolder('gestos/train', transform=train_transforms)
valid_dataset = ImageFolder('gestos/valid', transform=valid_transforms)
test_dataset = ImageFolder('gestos/test', transform=test_transforms)

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=32)
valid_loader = DataLoader(valid_dataset, shuffle=False, batch_size=128)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=256)

dataloaders_dict = {"train": train_loader, "valid": valid_loader}

# **Consideraciones para la carga del dataset**

*   Como las imágenes son de tamaño 200x200, hacemos un Resize para agrandar la imagen (los modelos pre-entrenados esperan una imagen de tamaño al menos 224x224).
*   Al conjunto de datos de entrenamiento es necesario darle un shuffle = True, ya que asi los datos los procesa de manera aleatoria en cada época.
* El tamaño del batch de entrenamiento tiene que ser bajo, para que el modelo tome en cada época pequeñas muestras de todo el conjunto.

# **Entrenamiento**

- Para el entrenamiento de nuestro modelos, utilizamos la funcion **train_model()** la cual recibe los parametro necesario para el entrenamiento. 

- Esta función internamente guarda en el directorio el mejor modelo de entrenamiento en base a la metrica **loss_valid**, dentro del ciclo de las épocas, así evitamos el sobre ajuste ya que nos quedamos con el mejor modelo entrenado y no con el último.


In [ ]:
def train_model(model, criterion, optimizer, num_epochs,best_model):
  best_valid_loss = np.inf
  for epoch in range(num_epochs):
      model.train()
      for x, y in train_loader:
          x=x.to(device)
          y=y.to(device)
          optimizer.zero_grad()
          yhat = model.forward(x)
          loss = criterion(yhat, y)
          loss.backward()
          optimizer.step()
      epoch_loss = 0.0
      model.eval()
      for x, y in valid_loader:
          x=x.to(device)
          y=y.to(device)
          yhat = model.forward(x)
          loss = criterion(yhat, y)
          epoch_loss += loss.item()
      if (epoch_loss < best_valid_loss):
        best_valid_loss = epoch_loss
        torch.save({'current_epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'current_valid_loss': epoch_loss
                   }, best_model)
        print("guardando..")
      print(epoch, epoch_loss)

# **Calculo de rendimiento**

In [ ]:
def testing(model):
  targets, predictions = [], []
  for mbdata, label in test_loader:
      mbdata, label = mbdata.to(device), label.to(device)
      logits = model.forward(mbdata)
      predictions.append(logits.argmax(dim=1).detach().cpu().numpy())     
      targets.append(label.cpu().numpy()) 
  predictions = np.concatenate(predictions) 
  targets = np.concatenate(targets)

  from sklearn.metrics import confusion_matrix, classification_report

  cm = confusion_matrix(targets, predictions)
  display(cm)
  print(classification_report(targets, predictions))

# **Modelos pre-entrenados**

A continuación usaremos modelos que nos provee [`torchvision.models`](https://pytorch.org/docs/stable/torchvision/models.html) para clasificar un conjunto de fotos en 0 dedos levantados, 1, 2, o 3. El proceso es el siguiente, primero se carga cada modelo, luego re-entrenaremos la última capa del modelo (en algunos casos puede ser más), para finalmente realizar las predicciones con el conjunto de datos de prueba, mostrando matrices de confusión, accuraccy y f1-score. Hay que tener en cuenta que cada modelo posee ciertos parámetros diferentes como la función de pérdida, el optimizador o el número de épocas, son esos parámetros los cuales podemos modificar para encontrar el adecuado en cada modelo.

# **Modelo ResNet**

In [ ]:
resnet = models.resnet18(pretrained=True, progress=True)

In [ ]:
#Congelamos todos los parámetros
for param in resnet.parameters(): 
    param.requires_grad = False

neuronsResnet = resnet.fc.in_features

In [ ]:
resnet.fc = torch.nn.Sequential(
                                  torch.nn.Linear(in_features=neuronsResnet, out_features=256, bias=True),
                                  torch.nn.ReLU(inplace=True), 
                                  torch.nn.Linear(in_features=256, out_features=128, bias=True),
                                  torch.nn.ReLU(inplace=True),
                                  torch.nn.Linear(in_features=128, out_features=4, bias=True))

resnet.to(device)

criterionResnet = torch.nn.CrossEntropyLoss()
optimizerResnet = torch.optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)
criterionResnet.to(device)
br = 'best_resnet.pt'

nEpochResnet = 10

**Entrenamiento del modelo**

In [ ]:
resnet.to(device)
train_model(resnet, criterionResnet, optimizerResnet, nEpochResnet,br)

guardando..
0 3.5243545286357403
guardando..
1 1.8489606939256191
guardando..
2 1.137607254087925
guardando..
3 0.8255511485040188
guardando..
4 0.6979451049119234
5 1.0714541701599956
6 0.7231131545267999
7 0.8753209854476154
8 0.8094796063378453
guardando..
9 0.6845975248143077


**Recuperando el mejor modelo**

In [ ]:
resnet = models.resnet18(pretrained=True)

resnet.fc = torch.nn.Sequential(
                                  torch.nn.Linear(in_features=neuronsResnet, out_features=256, bias=True),
                                  torch.nn.ReLU(inplace=True),
                                  torch.nn.Linear(in_features=256, out_features=128, bias=True),
                                  torch.nn.ReLU(inplace=True),  
                                  torch.nn.Linear(in_features=128, out_features=4, bias=True))
resnet.to(device)
resnet.load_state_dict(torch.load('best_resnet.pt')['model_state_dict'])

<All keys matched successfully>

**Matriz de confusión y reporte de clasificación**

In [ ]:
resnet.to(device)
testing(resnet)

array([[29,  0,  1,  0],
       [ 0, 30,  0,  0],
       [ 0,  0, 24,  6],
       [ 0,  0,  2, 28]])

              precision    recall  f1-score   support

           0       1.00      0.97      0.98        30
           1       1.00      1.00      1.00        30
           2       0.89      0.80      0.84        30
           3       0.82      0.93      0.87        30

    accuracy                           0.93       120
   macro avg       0.93      0.93      0.93       120
weighted avg       0.93      0.93      0.93       120



# **Observaciones de modelo resnet18**

*   Inicialmente usamos el optimizador Adam que nos proporciona torch.optim,obteniendo resultados relativamente malos (entre 0.50 y 0.70 en f1-score), cambiamos varias veces la tasa de aprendizaje, pero aun asi cambiaba poco los resultados, seguramente porque este tipo de red es muy grande y Adam va mejor en modelos que sean mas rápidos. Por este motivo cambiamos al optimizador SGD lo cual mejoró consireblemente los resultados como se ve en la última matriz de confusión.

*   Para la función de pérdida usamos la CrossEntropyLoss que une nn.LogSoftmax() y nn.NLLLoss() de torch, la cual es útil cuando se entrena un problema de clasificación como es en este caso.

* En cuanto a la arquitectura de la última capa, agregamos 2 capas extras y cada una con una función de activación Relu para disminuir el error provocado en cada capa.

# **Modelo AlexNet**

In [ ]:
alexnet = models.alexnet(pretrained=True, progress=True)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth


In [ ]:
for param in alexnet.parameters(): 
    param.requires_grad = False

alexnet.classifier = torch.nn.Sequential(torch.nn.Linear(9216, out_features=4000, bias=True),
                                  torch.nn.ReLU(inplace=True),
                                  torch.nn.Linear(in_features=4000, out_features=1000, bias=True),
                                  torch.nn.ReLU(inplace=True), 
                                  torch.nn.Linear(in_features=1000, out_features=4, bias=True))
alexnet.to(device)

criterionAlexNet = torch.nn.CrossEntropyLoss()
optimizerAlexNet = torch.optim.Adam(alexnet.parameters(), lr=0.0001)
criterionAlexNet.to(device)

nEpochAlexNet = 10
ba = 'best_alexnet.pt'
#display(alexnet)

**Entrenamiento del modelo**

In [ ]:
alexnet.to(device)
train_model(alexnet, criterionAlexNet, optimizerAlexNet, nEpochAlexNet, ba)

guardando..
0 11.780551041179024
guardando..
1 5.116900558542284
guardando..
2 4.207319953700534
guardando..
3 3.9937539512333657
guardando..
4 3.7068496449045334
guardando..
5 3.7047911156503797
guardando..
6 3.589061785658579
guardando..
7 3.5816695839970194
8 3.60148520372233
9 3.5840233730132436


**Recuperando el mejor modelo**

In [ ]:
best_alexnet = models.alexnet()

best_alexnet.classifier = torch.nn.Sequential(torch.nn.Linear(9216, out_features=4000, bias=True),
                                  torch.nn.ReLU(inplace=True),  
                                  torch.nn.Linear(in_features=4000, out_features=1000, bias=True),
                                  torch.nn.ReLU(inplace=True),  
                                  torch.nn.Linear(in_features=1000, out_features=4, bias=True))
best_alexnet.load_state_dict(torch.load('best_alexnet.pt')['model_state_dict'])
best_alexnet.to(device)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Linear(in_features=9216, out_features=4000, bias=True)
    (1): ReLU(inplace=True)
    (2): Lin

**Matriz de confusión y reporte de clasificación**

In [ ]:
best_alexnet.to(device)
testing(alexnet)

array([[14,  5,  2,  9],
       [ 0, 17,  3, 10],
       [ 0,  0, 21,  9],
       [ 0,  2,  5, 23]])

              precision    recall  f1-score   support

           0       1.00      0.47      0.64        30
           1       0.71      0.57      0.63        30
           2       0.68      0.70      0.69        30
           3       0.45      0.77      0.57        30

    accuracy                           0.62       120
   macro avg       0.71      0.62      0.63       120
weighted avg       0.71      0.62      0.63       120



# **Observaciones modelo AlexNet**

* La arquitectura de este modelo es similar a la Lenet5, pero con
mas filtros por capa y mas profunda. A la última capa le agreamos 2 capas adicionales que soportan una mayor cantidad de neuronas que la Lenet, para que la red sea mas compleja y cada una con una función de activación Relu para disminuir el error en cada capa.

* Para la función de pérdida usamos la misma que en el modelo anterior la CrossEntropyLoss, ya que es la que mejor funciona con problemas de clasificación.

* El optimizador en este caso usamos Adam con una tasa de aprendizaje de 0.0001 ya que en este caso posee menos capas ocultas en comparacion a la resnet y nos daba mejores resultados que el optimizador SGD

# **Modelo MobileNet v2**

In [ ]:
mobilenet = models.mobilenet_v2(pretrained=True)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


In [ ]:
for param in mobilenet.parameters(): 
    param.requires_grad = False

mobilenet.classifier = torch.nn.Sequential(torch.nn.Linear(1280, out_features=600, bias=True),
                                  torch.nn.ReLU(inplace=True),
                                  torch.nn.Linear(in_features=600, out_features=100, bias=True),
                                  torch.nn.ReLU(inplace=True),
                                  torch.nn.Linear(in_features=100, out_features=4, bias=True))
mobilenet.to(device)

criterionMobileNet = torch.nn.CrossEntropyLoss()
optimizerMobileNet = torch.optim.Adam(mobilenet.parameters(), lr=1e-3)
criterionMobileNet.to(device)

nEpochMobileNet = 10
bm = 'best_mobilenet.pt'

In [ ]:
mobilenet.to(device)
train_model(mobilenet, criterionMobileNet, optimizerMobileNet, nEpochMobileNet, bm)

guardando..
0 23.166823682375252
guardando..
1 20.45856876628818
2 30.907916438505254
3 35.13937383352095
4 62.415424939899985
5 27.91690421430419
6 73.26077154718949
7 37.166377453580594
8 49.56716681255145
9 68.83380017747083


**Recuperando el mejor modelo**

In [ ]:
best_mobilenet = models.mobilenet_v2(pretrained=True)
best_mobilenet.classifier = torch.nn.Sequential(torch.nn.Linear(1280, out_features=600, bias=True),
                                  torch.nn.ReLU(inplace=True),
                                  torch.nn.Linear(in_features=600, out_features=100, bias=True),
                                  torch.nn.ReLU(inplace=True),  
                                  torch.nn.Linear(in_features=100, out_features=4, bias=True))
best_mobilenet.load_state_dict(torch.load('best_mobilenet.pt')['model_state_dict'])
best_mobilenet.to(device)

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

**Matriz de confusión y reporte de clasificación**

In [ ]:
best_mobilenet.to(device)
testing(best_mobilenet)

array([[18,  4,  1,  7],
       [ 0, 22,  2,  6],
       [ 0,  7, 15,  8],
       [ 0,  3,  4, 23]])

              precision    recall  f1-score   support

           0       1.00      0.60      0.75        30
           1       0.61      0.73      0.67        30
           2       0.68      0.50      0.58        30
           3       0.52      0.77      0.62        30

    accuracy                           0.65       120
   macro avg       0.70      0.65      0.65       120
weighted avg       0.70      0.65      0.65       120



# **Observaciones MobileNet**

 * MobileNet v2 utiliza convoluciones ligeras en profundidad para filtrar
 entidades en la capa de expansión intermedia. Además, se eliminaron
 las no linealidades en las capas estrechas para mantener la potencia
 de representación. A la última capa le agregamos 2 cada una con 
una función de activación Relu.

* Para la función de pérdida usamo la CrossEntropyLoss siguiendo 
las mismas consideraciones que en los modelos anteriores

* El optimizador que elegimos al final fue Adam ya que era el que 
mejor resultados nos daba, teniendo una tasa de aprendizaje de 0.001. 

# **Modelo GoogleNet**

In [ ]:
googlenet = models.googlenet(pretrained=True, progress=True)

Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth


In [ ]:
for param in googlenet.parameters(): 
    param.requires_grad = False

In [ ]:
googlenet.fc = torch.nn.Sequential(torch.nn.Linear(in_features=1024, out_features=512, bias=True),
                                  torch.nn.ReLU(inplace=True),
                                  torch.nn.Linear(in_features=512, out_features=4, bias=True))

googlenet.to(device)

criterionGoogleNet = torch.nn.CrossEntropyLoss()
optimizerGoogleNet = torch.optim.SGD(googlenet.parameters(), lr=0.001, momentum=0.9)
criterionGoogleNet.to(device)  

nEpochGoogleNet = 10
bg = 'best_googlenet.pt'

**Entrenamiento del modelo**

In [ ]:
googlenet.to(device)
train_model(googlenet, criterionGoogleNet, optimizerGoogleNet, nEpochGoogleNet, bg)

guardando..
0 10.634019777178764
guardando..
1 7.710035875439644
guardando..
2 5.86761874333024
guardando..
3 5.833686497062445
guardando..
4 4.690093219280243
5 5.087892876937985
guardando..
6 4.404778080061078
7 4.907697562128305
8 5.154789896681905
9 4.941428128629923


**Recuperando el mejor modelo**

In [ ]:
best_googlenet = models.googlenet(pretrained=True)
best_googlenet.fc = torch.nn.Sequential(torch.nn.Linear(in_features=1024, out_features=512, bias=True),
                                  torch.nn.ReLU(inplace=True),
                                  torch.nn.Linear(in_features=512, out_features=4, bias=True))

best_googlenet.load_state_dict(torch.load('best_googlenet.pt')['model_state_dict'])
best_googlenet.to(device)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

**Matriz de confusión y reporte de clasificación**

In [ ]:
best_googlenet.to(device)
testing(best_googlenet)

array([[25,  1,  1,  3],
       [ 1, 19,  2,  8],
       [ 0,  6, 18,  6],
       [ 0,  1,  6, 23]])

              precision    recall  f1-score   support

           0       0.96      0.83      0.89        30
           1       0.70      0.63      0.67        30
           2       0.67      0.60      0.63        30
           3       0.57      0.77      0.66        30

    accuracy                           0.71       120
   macro avg       0.73      0.71      0.71       120
weighted avg       0.73      0.71      0.71       120



# **Observaciones GoogleNet**

* GoogLeNet se basó en una arquitectura de red neuronal convolucional profunda con el nombre en código "Inception", que era responsable de establecer el nuevo estado de la técnica para la clasificación y detección en el ImageNet Large-Scale Visual Recognition Challenge 2014. Este modelo pre-entrenado que tiene una arquitectura bastante compleja, decidimos agregar una capa Linear conectada a la ultima caoa de salida, de acuerdo a nuestro problema propio.

* En cuanto a la función de perdida, decidimos utilizar CrossEntropyLoss siguiendo las mismas consideraciones que los modelos.

* Para el optimizador cambiamos de Adam a SGD, ya que funciona mejor con redes mas complejas y grandes como es en este caso, pero al probar con ambos optimizadores, los resultados eran bastante similares.

# **Conclusiones Generales**

Nos dimos cuenta que cada modelo funciona diferente, y los hiper-parámetreos que uno modifica, no son siempre los mismos valores. Para cada problema pueden variar estos parámetros, y por eso es sumamente necesario saber como funcionan los modelos y a que corresponde cada parámetro. En este caso, los mejores resultados los obtuvimos con el modelo resnet18, que en comparación a los otros 3 modelos sus resultados vistos en la matriz de confusión son mejores, mas precisas y con un mejor recall. La razón de estos resultados puede deberse a que el modelo resnet es uno de los más recientes y por lo tanto tiene mejoras en su algoritmo de redes neuronales que otros modelos mas antiguos.

# **Resnet18 vs Lenet5 modificada**

In [ ]:
train_transformsL = transforms.Compose([#transforms.Resize(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

valid_transformsL = transforms.Compose([#transforms.Resize(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])


test_transformsL = transforms.Compose([#transforms.Resize(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])


train_datasetL = ImageFolder('gestos/train', transform=train_transformsL)
valid_datasetL = ImageFolder('gestos/valid', transform=valid_transformsL)
test_datasetL = ImageFolder('gestos/test', transform=test_transformsL)

train_loaderL = DataLoader(train_datasetL, shuffle=True, batch_size=32)
valid_loaderL = DataLoader(valid_datasetL, shuffle=False, batch_size=256)
test_loaderL = DataLoader(test_datasetL, shuffle=True, batch_size=512)

In [ ]:
def train_lenet5(model, criterion, optimizer, num_epochs, best_model):
  best_valid_loss = np.inf
  for epoch in range(num_epochs):
      model.train()
      for x, y in train_loaderL:
          x=x.to(device)
          y=y.to(device)
          optimizer.zero_grad()
          yhat = model.forward(x)
          loss = criterion(yhat, y)
          loss.backward()
          optimizer.step()
      epoch_loss = 0.0
      model.eval()
      for x, y in valid_loaderL:
          x=x.to(device)
          y=y.to(device)
          yhat = model.forward(x)
          loss = criterion(yhat, y)
          epoch_loss += loss.item()
      if (epoch_loss < best_valid_loss):
        best_valid_loss = epoch_loss
        torch.save({'current_epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'current_valid_loss': epoch_loss
                   }, best_model)
        print("guardando..")
      print(epoch, epoch_loss)

In [ ]:
def testingL(model):
  targets, predictions = [], []
  for mbdata, label in test_loaderL:
      mbdata, label = mbdata.to(device), label.to(device)
      logits = model.forward(mbdata)
      predictions.append(logits.argmax(dim=1).detach().cpu().numpy())     
      targets.append(label.cpu().numpy()) 
  predictions = np.concatenate(predictions) 
  targets = np.concatenate(targets)

  from sklearn.metrics import confusion_matrix, classification_report

  cm = confusion_matrix(targets, predictions)
  display(cm)
  print(classification_report(targets, predictions))

In [ ]:
import torch

class Lenet5(torch.nn.Module):

    def __init__(self):
        super(type(self), self).__init__()
        self.conv1 = torch.nn.Conv2d(kernel_size = 5, in_channels = 3, out_channels = 6)
        self.conv2 = torch.nn.Conv2d(kernel_size = 5, in_channels = 6, out_channels = 16)
        self.conv3 = torch.nn.Conv2d(kernel_size = 5, in_channels = 16, out_channels = 32)
        self.conv4 = torch.nn.Conv2d(kernel_size = 5, in_channels = 32, out_channels = 64)
        self.mpool = torch.nn.MaxPool2d(kernel_size=2)
        self.activation = torch.nn.ReLU()
        self.linear1 = torch.nn.Linear(in_features=64*8*8,out_features=800)
        self.linear2 = torch.nn.Linear(in_features = 800, out_features =400)
        self.linear3 = torch.nn.Linear(in_features = 400, out_features =200)
        self.linear4 = torch.nn.Linear(in_features = 200, out_features =100)
        self.linear5 = torch.nn.Linear(in_features=100,out_features=4)
    def forward(self, x):
        h = self.mpool(self.activation(self.conv1(x)))
        h = self.mpool(self.activation(self.conv2(h)))
        h = self.mpool(self.activation(self.conv3(h)))
        h = self.mpool(self.activation(self.conv4(h)))
        #print(h.shape)
        h=h.view(-1, 64*8*8)
        h = self.activation(self.linear1(h))
        h = self.activation(self.linear2(h))
        h = self.activation(self.linear3(h))
        h = self.activation(self.linear4(h))


        return self.linear5(h)

model = Lenet5()
display(model)

Lenet5(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (mpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (activation): ReLU()
  (linear1): Linear(in_features=4096, out_features=800, bias=True)
  (linear2): Linear(in_features=800, out_features=400, bias=True)
  (linear3): Linear(in_features=400, out_features=200, bias=True)
  (linear4): Linear(in_features=200, out_features=100, bias=True)
  (linear5): Linear(in_features=100, out_features=4, bias=True)
)

In [ ]:
model.to(device)
nEpochLeNet5 = 10


optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss()
criterion.to(device) 

bl = 'best_model'

In [ ]:
train_lenet5(model, criterion, optimizer, nEpochLeNet5, bl)

guardando..
0 45.19896803796291
1 55.31865841895342
2 95.57890623807907
3 65.38756975531578
4 83.97207576036453
5 77.37592155113816
6 63.682384757790715
7 167.3541099005961
8 45.877446696627885
9 123.5543510497555


In [ ]:
best_model = Lenet5()

best_model.load_state_dict(torch.load('best_model')['model_state_dict'])
best_model.to(device)

Lenet5(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (mpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (activation): ReLU()
  (linear1): Linear(in_features=4096, out_features=800, bias=True)
  (linear2): Linear(in_features=800, out_features=400, bias=True)
  (linear3): Linear(in_features=400, out_features=200, bias=True)
  (linear4): Linear(in_features=200, out_features=100, bias=True)
  (linear5): Linear(in_features=100, out_features=4, bias=True)
)

In [ ]:
testingL(best_model)

array([[ 3, 18,  5,  4],
       [ 0, 29,  1,  0],
       [ 0, 18,  7,  5],
       [ 0, 16,  8,  6]])

              precision    recall  f1-score   support

           0       1.00      0.10      0.18        30
           1       0.36      0.97      0.52        30
           2       0.33      0.23      0.27        30
           3       0.40      0.20      0.27        30

    accuracy                           0.38       120
   macro avg       0.52      0.38      0.31       120
weighted avg       0.52      0.38      0.31       120



# **Comparación resnet18 vs lenet5 modificada**

En la comparación entre estos dos modelos, claramente el nesnet18 es muy superior considerando que es un modelo preentrenado y el lenet5 no. La arquitectura es mas compleja y provada en el resnet 18 y esto lo hace tener mejores resultados.

- En cuanto a **precisión** resnet tiene valores (0.98, 1.0, 0.89, 0.82) para las diferentes clases, lo cuál nos muestra valores casi perfectos en comparación con la lenet5  (1.0, 0.36, 0.33, 0.40) que sola la primera clase tiene un buen valor y las demas bordean el 38% de precision.

- En cuanto al **recall**, la resnet18 sigue con numeros en porcentaje bastante alto. En cambio la lenet5 (0.1, 0.97, 0.23, 0.20) tenemos valores bastate malos como por ejemplo en la primera clase (0.1), lo cual no sa indicio de que el modelo no es lo sufuciente mente robusto para resolver el problema.

- Para la métrica **f1-score**, tenemos valores en torno a 0.90 en la resnet18 para todas las clases. Para lenet5, seguimos viendo peores números que rondan los 0.3

Hay que tener en cuenta que el modelo que creamos estaba sobre-entrenado, ya que al ver los valores de la loss en cada época nunca bajo más después de la primera época, también hicimos la prueba de "testear" el conjunto de entrenamiento, para el que obtuvimos resultados muy buenos lo cual no era consistente con los resultados obtenidos con el conjunto de "test", intentamos aplicar aumentación de datos, pero no logramos evitar el sobre ajuste y menos aún mejorar los resultados.

En conclución como se esperaba, la resnet18 pre-entrenada fue muy superior a la lenet5 modificada, considerando obviamente la complejidad de las dos arquitecturas. 

